# Dihedral symmetry of the Robinson form

**Adapted from**: Example 5.4 of [GP04]

[GP04] Gatermann, Karin and Parrilo, Pablo A.
*Symmetry groups, semidefinite programs, and sums of squares*.
Journal of Pure and Applied Algebra 192.1-3 (2004): 95-128.

Symmetry reduction is still a work in progress in SumOfSquares, so we include the following files that will be incorporated into SumOfSquares.jl once SymbolicWedderburn.jl is released:

In [1]:
using SumOfSquares
include(joinpath(dirname(dirname(pathof(SumOfSquares))), "examples", "symmetry.jl"))
include(joinpath(dirname(dirname(pathof(SumOfSquares))), "examples", "scaled_perm.jl"))

    Cloning git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Project.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/nonperm`
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Manifest.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#bl/nonperm`


We start by defining the Dihedral group of order 8.
This group is isomorphic to the following permutation group:

In [2]:
d = perm"(1, 2, 3, 4)"
c = perm"(1, 3)"
G = PermGroup([c, d])

Permutation group on 2 generators generated by
 (1,3)
 (1,2,3,4)

We could rely on this isomorphism to define this group.
However, in order to illustrate how to do symmetry reduction with a custom group,
we show in this example what should be implemented to define a new group.

In [3]:
struct DihedralElement <: GroupElem
    n::Int
    reflection::Bool
    id::Int
end
function PermutationGroups.order(el::DihedralElement)
    if el.reflection
        return 2
    else
        if iszero(el.id)
            return 1
        else
            return div(el.n, gcd(el.n, el.id))
        end
    end
end
Base.one(el::Union{DihedralElement}) = DihedralElement(el.n, false, 0)
function Base.inv(el::DihedralElement)
    if el.reflection || iszero(el.id)
        return el
    else
        return DihedralElement(el.n, false, el.n - el.id)
    end
end
function PermutationGroups.mul!(::DihedralElement, a::DihedralElement, b::DihedralElement)
    a.n == b.n || error("Cannot multiply elements from different Dihedral groups")
    id = mod(a.reflection ? a.id - b.id : a.id + b.id, a.n)
    return DihedralElement(a.n, a.reflection != b.reflection, id)
end
function Base.:^(el::DihedralElement, k::Integer)
    if el.reflection
        return iseven(k) ? one(el) : el
    else
        return DihedralElement(el.n, false, mod(el.id * k, el.n))
    end
end

struct DihedralGroup <: Group
    n::Int
end
_orbit(cc::Vector{<:GroupElem}) = PermutationGroups.Orbit(cc, Dict(a => nothing for a in cc))
_orbit(el::GroupElem) = _orbit([el])
function SymbolicWedderburn.conjugacy_classes_orbit(d::DihedralGroup)
    orbits = [_orbit(DihedralElement(d.n, false, 0))]
    for i in 1:div(d.n - 1, 2)
        push!(orbits, _orbit([
            DihedralElement(d.n, false, i),
            DihedralElement(d.n, false, d.n - i),
        ]))
    end
    if iseven(d.n)
        push!(orbits, _orbit(DihedralElement(d.n, false, div(d.n, 2))))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:2:(d.n - 2)
        ]))
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 1:2:(d.n - 1)
        ]))
    else
        push!(orbits, _orbit([
            DihedralElement(d.n, true, i) for i in 0:(d.n - 1)
        ]))
    end
end

The Robinson form is invariant under the following action of the Dihedral group on monomials:
The action of each element of the groups is to map the variables `x, y` to:
| id | rotation | reflection |
|----|----------|------------|
| 0  | x, y     | y, x       |
| 1  | -y, x    | -x, y      |
| 2  | -x, -y   | -y, -x     |
| 3  | y, -x    | x, -y      |

In [4]:
using DynamicPolynomials
@polyvar x y
function action(mono::MP.AbstractMonomial, el::DihedralElement)
    if iseven(el.reflection + el.id)
        var_x, var_y = x, y
    else
        var_x, var_y = y, x
    end
    sign_x = 1 <= el.id <= 2 ? -1 : 1
    sign_y = 2 <= el.id ? -1 : 1
    return MP.substitute(MP.Eval(), mono, [x, y] => [sign_x * var_x, sign_y * var_y])
end
function action(term::MP.AbstractTerm, el::DihedralElement)
    return MP.coefficient(term) * action(MP.monomial(term), el)
end
function action(poly::MP.AbstractPolynomial, el::DihedralElement)
    return MP.polynomial([action(term, el) for term in MP.terms(poly)])
end

poly = x^6 + y^6 - x^4 * y^2 - y^4 * x^2 - x^4 - y^4 - x^2 - y^2 + 3x^2 * y^2 + 1

x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1

We can verify that `poly` is indeed invariant under each element of the group as follows.

In [5]:
G = DihedralGroup(4)
for cc in SymbolicWedderburn.conjugacy_classes_orbit(G)
    for g in cc
        @show action(poly, g)
        @show action(poly, g) == poly
    end
end

action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true
action(poly, g) = x⁶ - x⁴y² - x²y⁴ + y⁶ - x⁴ + 3x²y² - y⁴ - x² - y² + 1
action(poly, g) == poly = true


We can exploit this symmetry for reducing the problem using the `SymmetricIdeal` certificate as follows:

In [6]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
con_ref = @constraint(model, poly - t in SOSCone(), ideal_certificate = SymmetricIdeal(SOSCone(), G, action))
optimize!(model)
value(t)

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.03e-01 Pobj: -4.5636299e+00 Ad: 8.45e-01 Dobj:  5.2967876e+00 
Iter:  2 Ap: 7.31e-01 Pobj: -1.1835354e+01 Ad: 8.76e-01 Dobj:  1.4675311e+01 
Iter:  3 Ap: 8.91e-01 Pobj: -6.8243431e+00 Ad: 7.62e-01 Dobj:  7.6214135e+00 
Iter:  4 Ap: 8.67e-01 Pobj: -2.7355189e+00 Ad: 8.59e-01 Dobj:  1.2146327e+00 
Iter:  5 Ap: 8.79e-01 Pobj: -1.5458374e+00 Ad: 8.36e-01 Dobj: -3.5074262e-01 
Iter:  6 Ap: 7.96e-01 Pobj: -1.1190240e+00 Ad: 7.94e-01 Dobj: -7.3112750e-01 
Iter:  7 Ap: 8.52e-01 Pobj: -9.9963334e-01 Ad: 7.39e-01 Dobj: -8.5858732e-01 
Iter:  8 Ap: 7.76e-01 Pobj: -9.5570165e-01 Ad: 7.75e-01 Dobj: -9.0656574e-01 
Iter:  9 Ap: 8.61e-01 Pobj: -9.4215903e-01 Ad: 7.57e-01 Dobj: -9.2437049e-01 
Iter: 10 Ap: 7.84e-01 Pobj: -9.3679565e-01 Ad: 8.32e-01 Dobj: -9.3072397e-01 
Iter: 11 Ap: 8.30e-01 Pobj: -9.3508080e-01 Ad: 8.13e-01 Dobj: -9.3273624e-01 
Iter: 12 Ap: 7.95e-01 Pobj: -9.3455269e-01 Ad: 8.23e-

-0.93338346876774

We indeed find `-3825/4096`, let's verify that symmetry was exploited:

In [7]:
for g in gram_matrix(con_ref).sub_gram_matrices
    println(g.basis.polynomials)
    display(g.Q)
end

DynamicPolynomials.Polynomial{true,Float64}[x³, x²y, xy², y³, x, y]
DynamicPolynomials.Polynomial{true,Float64}[x² + y², 1.0]
DynamicPolynomials.Polynomial{true,Float64}[xy]
DynamicPolynomials.Polynomial{true,Float64}[x² - y²]


6×6 SymMatrix{Float64}:
  1.0        0.0       -0.999879   0.0       -0.625038   0.0
  0.0        0.999757   0.0       -0.999879   0.0        0.624962
 -0.999879   0.0        0.999757   0.0        0.624962   0.0
  0.0       -0.999879   0.0        1.0        0.0       -0.625038
 -0.625038   0.0        0.624962   0.0        0.390672   0.0
  0.0        0.624962   0.0       -0.625038   0.0        0.390672

2×2 SymMatrix{Float64}:
  0.250076  -0.695336
 -0.695336   1.93338

1×1 SymMatrix{Float64}:
 6.208958098406713e-10

1×1 SymMatrix{Float64}:
 5.355378711438148e-10

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*